In [1]:
import re
import numpy as np
import pandas as pd

# 
def preprocess_sentence_code(sentence: str, action_words):
    '''
    '''
    # 去除 '.'
    sentence = sentence.replace('.', '')
    # 多个空格变一个空格
    sentence = ' '.join(sentence.split())
    # 大写化
    # 判断是否多动词共有主语
    match = re.search(r".*?(?={0}) .*(AND?={0})*".format(action_words), sentence, flags=re.I)
    if match:
        # 找出主语
        entity = re.search(r"(?P<entity>.*?)"+r"(?={0})".format(action_words), sentence, flags=re.I)
        entity = entity.group("entity")
        # 分割原句 
        sentence_split = re.sub(r" AND(?={0})".format(action_words), "|", sentence)
        sentence_ls = sentence_split.split('|')
        # 主语归位
        res_sentence_ls = []
        for i, s in enumerate(sentence_ls):
            if i == 0:
                res_sentence_ls.append(s)
            else:
                res_sentence_ls.append(entity + s)
        return res_sentence_ls
    else:
        return [sentence]

# sentence = "RWY 12 NOT AVBL DUE WIP EXC SNOW AND U/S DUE TO RAIN AND U/S" 
# action_words = " NOT AVBL| U/S"
# preprocess_sentence_code(sentence, action_words)

# 
def read_words(path, sheet_name='words_list'):
    '''
    '''
    df_words = pd.read_excel(path, sheet_name=sheet_name)
    verb_ls, limit_words_ls = set(df_words['ACTION'].values), set(df_words['LIMIT'].values)
    verb_ls, limit_words_ls = [word for word in verb_ls if isinstance(word, str)], [word for word in limit_words_ls if isinstance(word, str)]
    # entity = "ILS RWY|RWY"
    action = " " + "| ".join(verb_ls)
    reason = " DUE"
    # limit = " EXCEPT | EXC | EXP | WHEN | FOLLOWING CONDITIONS | ONLY | FLW LIMITATIONS | IN CASE OF "
    limit = " " + "| ".join(limit_words_ls)
    source = " REFER| REF"
    return action, reason, limit, source

# 
def get_item_pattern_list(action_words, reason_words, limit_words, source_words, path, sheet_name='base_rules', ):
    '''
    '''
    df_rules = pd.read_excel(path, sheet_name=sheet_name)
    format_ls = list(set(df_rules['FORMAT'].values))
    pattern_entity_ls, pattern_action_ls, pattern_reason_ls, pattern_limit_ls, pattern_source_ls = [], [], [], [], []
    for item in format_ls:
        if item == "entity":
            pattern_entity_ls = list(df_rules.loc[df_rules['FORMAT'] == item, 'RULES'].values)
        elif item == "action":
            pattern_action_ls = list(df_rules.loc[df_rules['FORMAT'] == item, 'RULES'].values)
        elif item == "reason":
            pattern_reason_ls = list(df_rules.loc[df_rules['FORMAT'] == item, 'RULES'].values)
        elif item == "limit":
            pattern_limit_ls = list(df_rules.loc[df_rules['FORMAT'] == item, 'RULES'].values)
        elif item == "source":
            pattern_source_ls = list(df_rules.loc[df_rules['FORMAT'] == item, 'RULES'].values)
        else:
            print("error")

    # pattern_entity_ls = [p.format(entity) for p in pattern_entity_ls]
    pattern_action_ls = [p.format(action_words) for p in pattern_action_ls]
    pattern_reason_ls = [p.format(reason_words) for p in pattern_reason_ls]
    pattern_limit_ls = [p.format(limit_words) for p in pattern_limit_ls]
    pattern_source_ls = [p.format(source_words) for p in pattern_source_ls]

    return pattern_entity_ls, pattern_action_ls, pattern_reason_ls, pattern_limit_ls, pattern_source_ls


# pattern combin
def pattern_combine(pattern_entity_ls, pattern_action_ls, pattern_reason_ls, pattern_limit_ls, pattern_source_ls):
    '''
    '''
    pattern_ls = []
    # 主语 + 动词 + 原因 + 限制 + 来源
    for p_entity in pattern_entity_ls:
        for pattern_action in pattern_action_ls:
            for p_reason in pattern_reason_ls:
                for p_limit in pattern_limit_ls:
                    for p_source in pattern_source_ls:
                        pattern_ls.append(p_entity + pattern_action + p_reason + p_limit + p_source)
    # 主语 + 动词 + 原因 + 限制
    for p_entity in pattern_entity_ls:
        for pattern_action in pattern_action_ls:
            for p_reason in pattern_reason_ls:
                for p_limit in pattern_limit_ls:
                    pattern_ls.append(p_entity + pattern_action + p_reason + p_limit)
    # 主语 + 动词 + 原因 + 限制
    for p_entity in pattern_entity_ls:
        for pattern_action in pattern_action_ls:
            for p_limit in pattern_limit_ls:
                for p_reason in pattern_reason_ls:
                    pattern_ls.append(p_entity + pattern_action + p_limit + p_reason)
    # 主语 + 动词 + 原因
    for p_entity in pattern_entity_ls:
        for pattern_action in pattern_action_ls:
            for p_reason in pattern_reason_ls:
                pattern_ls.append(p_entity + pattern_action + p_reason)
    # 主语 + 动词 + 限制
    for p_entity in pattern_entity_ls:
        for pattern_action in pattern_action_ls:
            for p_limit in pattern_limit_ls:
                pattern_ls.append(p_entity + pattern_action + p_limit)
    # 主语 + 动词 + 来源
    for p_entity in pattern_entity_ls:
        for pattern_action in pattern_action_ls:
            for p_source in pattern_source_ls:
                pattern_ls.append(p_entity + pattern_action + p_source)
    # 主语 + 动词
    pattern_action_ls_greedy = [s[:-2] + ')' for s in pattern_action_ls] # to greedy model
    for p_entity in pattern_entity_ls:
        for pattern_action in pattern_action_ls_greedy:
            pattern_ls.append(p_entity + pattern_action)
    
    return pattern_ls 


#
def get_general_rules(path, words_sheet, rules_sheet):
    '''
    '''
    # read_words
    action_words, reason_words, limit_words, source_words = read_words(path=path, sheet_name=words_sheet)
    # get each item pattern list
    pattern_entity_ls, pattern_action_ls, pattern_reason_ls, pattern_limit_ls, pattern_source_ls = get_item_pattern_list(action_words, reason_words, limit_words, source_words, path=path, sheet_name=rules_sheet)
    # pattern combine 
    pattern_ls = pattern_combine(pattern_entity_ls, pattern_action_ls, pattern_reason_ls, pattern_limit_ls, pattern_source_ls)
    return pattern_ls

# RULES_TABLE = 'E:/Workstation/data/NOTAM/NOTAM_table.xlsx'
# GENERAL_RULES = get_general_rules(RULES_TABLE, words_sheet="words_list", rules_sheet="base_rules")
# len(GENERAL_RULES)
# pattern add
# supplement_rules = pd.read_excel(RULES_TABLE, sheet_name="supplement_rules")['RULES'].values.tolist()
    
# supplement_rules.extend(pattern_ls)
# pattern_ls = supplement_rules

# pattern add
def get_supplement_rules(path, words_sheet, rules_sheet):
    '''
    '''
    # read_words
    action_words, reason_words, limit_words, source_words = read_words(path=path, sheet_name=words_sheet)
    # read supplement_rules
    df_rules = pd.read_excel(path, sheet_name=rules_sheet)
    format_ls = list(set(df_rules['FORMAT'].values))
    # print(format_ls)
    supplement_pattern_ls = []
    for item in format_ls:
        if item == "NONE":
            supplement_pattern_ls.extend(list(df_rules.loc[df_rules['FORMAT'] == item, 'RULES'].values))
        elif item == "action":
            supplement_pattern_ls.extend([rule.format(action_words) for rule in list(df_rules.loc[df_rules['FORMAT'] == item, 'RULES'].values)])
        elif item == "reason":
            supplement_pattern_ls.extend([rule.format(reason_words) for rule in list(df_rules.loc[df_rules['FORMAT'] == item, 'RULES'].values)])
        elif item == "limit":
            supplement_pattern_ls.extend([rule.format(limit_words) for rule in list(df_rules.loc[df_rules['FORMAT'] == item, 'RULES'].values)])
        elif item == "source":
            supplement_pattern_ls.extend([rule.format(source_words) for rule in list(df_rules.loc[df_rules['FORMAT'] == item, 'RULES'].values)])
        else:
            print("error")
    return supplement_pattern_ls

# RULES_TABLE = 'E:/Workstation/data/NOTAM/NOTAM_table.xlsx'
# SUPPLEMENT_RULES = get_supplement_rules(RULES_TABLE, words_sheet="words_list", rules_sheet="supplement_rules")
# len(SUPPLEMENT_RULES)


In [3]:
RULES_TABLE = 'E:/Workstation/data/NOTAM/NOTAM_table.xlsx'
GENERAL_RULES = get_general_rules(RULES_TABLE, words_sheet="words_list", rules_sheet="base_rules")
SUPPLEMENT_RULES = get_supplement_rules(RULES_TABLE, words_sheet="words_list", rules_sheet="supplement_rules")
SUPPLEMENT_RULES.extend(GENERAL_RULES)
RULES_LIST = SUPPLEMENT_RULES
print(len(RULES_LIST))

32


In [4]:
RULES_TABLE = 'E:/Workstation/data/NOTAM/NOTAM_table.xlsx'
GENERAL_RULES = get_general_rules(RULES_TABLE, words_sheet="words_list", rules_sheet="base_rules")
SUPPLEMENT_RULES = get_supplement_rules(RULES_TABLE, words_sheet="words_list", rules_sheet="supplement_rules")
SUPPLEMENT_RULES.extend(GENERAL_RULES)
RULES_LIST = SUPPLEMENT_RULES
print(len(RULES_LIST))

# sentence_parse
def sentence_parse(sentence_code: str):
    '''
    '''
    
    # preprocess sentence_code
    # read_words
    action_words, _, _, _ = read_words(path=RULES_TABLE, sheet_name="words_list")
    sentence_ls = preprocess_sentence_code(sentence_code, action_words)
    # sentence_ls = [sentence_code]
    
    # sentence_parse
    is_match = False
    res_list_ls = []
    for sentence in sentence_ls:
        tmp_is_match = False
        res_dict = {item: "" for item in ['entity', 'runway', 'action', 'reason', 'limit', 'limit_wings', 'limit_weight', 'source']} 
        for pattern in RULES_LIST:
            match = re.search(pattern, sentence, flags=re.I)
            if match:
                tmp_is_match = True
                match_dict = match.groupdict()
                # print(match_dict)
                if 'entity' in match_dict:
                    res_dict['entity'] = match_dict['entity']
                if 'entity_supply' in match_dict:
                    res_dict['entity'] = res_dict['entity'] + ' ' + match_dict['entity_supply']
                if 'runway' in res_dict:
                    res_dict['runway'] = str(re.findall(r"RWY [0-9]{0,2}[LR]?/?[0-9]{0,2}[LR]?|RUNWAY [0-9]{0,2}[LR]?/?[0-9]{0,2}[LR]?", res_dict['entity'], flags=re.I))
                if 'action' in match_dict:
                    res_dict['action'] = match_dict['action']
                if 'reason' in match_dict:
                    res_dict['reason'] = match_dict['reason']
                if 'limit' in match_dict:
                    res_dict['limit'] = match_dict['limit']
                if 'source' in match_dict:
                    res_dict['source'] = match_dict['source']
                # print(res_dict)
                res_list = list(res_dict.values())
                res_list = [item.strip() for item in res_list]
                # print(res_list)
                res_list_ls.append(res_list)
                break
                # return (is_match, [res_list])
        is_match = is_match or tmp_is_match
    return (is_match, res_list_ls)

# test
txt = ["RWY 12/30 U/S FOR LDG",
       "RUNWAY IMMEDIATELY AVAILABLE IN CASE OF AN EMERGENCY",
       "RWY 13/31 USABLE FOR HELICOPTERS ONLY",
       "RWY NOT AVBL FOR OPS DUE REPAIR WIP",
       "RWY 32C AVBL ONLY FOR . ACFT DEP",
       "ACFT WITH MTOW 25 TONS OR ABOVE MUST PERFORM 180 DEG TURN ON TURN",
       "RWY 17 CLSD",
       "DUE TO...",
       "RWY 30 AVBL FOR TKOF AND LDG UNDER. FOLLOWING CONDITIONS:X-WIND COMPONENT ON MAIN RWY S EXCEEDS 15 KTS AND ATC IS VISUAL WITH BOTH RWY ENDS RWY 12/30.",
       "DUE TO WIP ON AUX POWER SUPPLY FOR RWY 12/30 SHORT-TERM POWER FAILURE PAPI RWY 12 AND SALS RWY 12 MAY BE EXPECTED.",
       "RWY 13/31 USABLE FOR HELICOPTERS ONLY",
       "RWY STRENGTH LOWERED - SOAKED SURFACE. ARR/DEP WITH PRIOR AD OPERATOR APV ONLY. CTC +420 777 000 494.",
       "RWY 06/24 AVBL PPR 30 MIN CTC 514-633-3488 EXC PISTON AND TURBOPROP ACFT MAY DEP FM TWY B6 WITHOUT PPR",
       "PAPI RWY 14/32 DO NOT USE AWAITING FLTCK REF MILAIP AD 2 LIBN 1-7",
       "THE FIRST 600M OF RWY 06R/24L AND TAXILING C2 WILL BE CLOSED TO ALL TRAFFIC DUE TO U.A.S UNIT DEPLOYMENT AND TRAINING FLIGHTS",
       "RWY 08/26 NARROWED TO 30M FROM SOUTHERN EDGE", # 
       "ILS RWY 05L ON FLIGHT CALIBRATION. DO NOT. USE, FALSE INDCATIONS POSS.",
       "RWY 06/24 LTD, BUMPS PRESENCE. EXER CTN",
       "RWY 10/28 DAMAGED BY WILD ANIMALS TKOF/LDG ACT WITH AD OPERATOR APV ONLY.",
       ]

for i, sentence in enumerate(txt):
    print(i, sentence_parse(sentence))


32
0 (True, [['RWY 12/30  FOR LDG', "['RWY 12/30']", 'U/S', '', '', '', '', '']])
1 (True, [['RUNWAY IMMEDIATELY', "['RUNWAY ']", 'AVAILABLE', '', 'IN CASE OF AN EMERGENCY', '', '', '']])
2 (True, [['RWY 13/31  FOR HELICOPTERS ONLY', "['RWY 13/31']", 'USABLE', '', '', '', '', '']])
3 (True, [['RWY  FOR OPS', "['RWY ']", 'NOT AVBL', 'DUE REPAIR WIP', '', '', '', '']])
4 (True, [['RWY 32C', "['RWY 32']", 'AVBL', '', 'ONLY FOR ACFT DEP', '', '', '']])
5 (True, [['ACFT WITH MTOW 25 TONS OR ABOVE MUST PERFORM 180 DEG  ON TURN', '[]', 'TURN', '', '', '', '', '']])
6 (True, [['RWY 17', "['RWY 17']", 'CLSD', '', '', '', '', '']])
7 (False, [])
8 (True, [['RWY 30  FOR TKOF AND LDG', "['RWY 30']", 'AVBL', '', 'UNDER FOLLOWING CONDITIONS:X-WIND COMPONENT ON MAIN RWY S EXCEEDS 15 KTS AND ATC IS VISUAL WITH BOTH RWY ENDS RWY 12/30', '', '', '']])
9 (True, [['RWY 12/30', "['RWY 12/30']", '', 'DUE TO WIP ON AUX POWER SUPPLY', 'SHORT-TERM POWER FAILURE PAPI RWY 12 AND SALS RWY 12 MAY BE EXPECTED', '',

In [5]:
# E项 无分句 test
test_file = "E:/Workstation/data/NOTAM/t.xlsx"
df = pd.read_excel(test_file)["E项"]
txt = df.values
txt = [s for s in txt if isinstance(s, str)]
res = []
for i, sentence in enumerate(txt): 
    res.append((sentence_parse(sentence)[1]))  
    # print(i, sentence_parse(sentence))
print(res)

[[['RWY 30  FOR TKOF AND LDG', "['RWY 30']", 'AVBL', '', 'UNDER FOLLOWING CONDITIONS:X-WIND COMPONENT ON MAIN RWY S EXCEEDS 15 KTS AND ATC IS VISUAL WITH BOTH RWY ENDS RWY 12/30', '', '', '']], [['RWY 11/29  ALL TRAINING AND VFR FLIGHTS', "['RWY 11/29']", 'CLOSED', '', '', '', '', '']], [['RWY 10R/28L', "['RWY 10R/28L']", 'STRENGTH DECREASED', '', 'SOAKED RWY ARR/DEP WITH AD OPERATOR APPROVAL ONLY TEL +420 602731152', '', '', '']], [['RWY 13/31', "['RWY 13/31']", 'NARROWED TO 22,5M', '', 'WHEN WET FOR LDG/TKOF USE SOUTHERNHALF OF RWY', '', '', '']], [['RWY 13/31  FOR HELICOPTERS ONLY', "['RWY 13/31']", 'USABLE', '', '', '', '', '']], [['REF AIP SUP 12/21 PARAS 23 - 24 AND NOTAM A1171/21,CLOSURE OF CENTRE RWY', '[]', 'IS CNL AND VHHH ON DUAL RWY OPS', 'DUE HKIA CARGO STAND RE-DESIGNATION', 'DRG 252316 -262315', '', '', '']], [['RWY 06R/24L', "['RWY 06R/24L']", 'CLSD', '', 'AVBL PPR 30MIN', '', '', '']], [['REDL 01/19', '[]', 'U/S', '', 'NIGHT OPS NOT AUTH', '', '', '']], [['EDGE LIGHTS,

In [6]:
entity_ls = [] # 0
runway_ls = [] # 1
action_ls = [] # 2
reason_ls = [] # 3
limit_ls = [] # 4
source_ls = [] # 7
for item in res:
    
    if item:
        item = item[0]
        entity_ls.append(item[0])
        runway_ls.append(item[1])
        action_ls.append(item[2])
        reason_ls.append(item[3])
        limit_ls.append(item[4])
        source_ls.append(item[7])
        print(item)
    else:
        entity_ls.append("")
        runway_ls.append("")
        action_ls.append("")
        reason_ls.append("")
        limit_ls.append("")
        source_ls.append("")

len(entity_ls), len(source_ls)

df = pd.DataFrame({"txt": txt, "entity": entity_ls, "runway": runway_ls, "action": action_ls, "reason": reason_ls, "limit": limit_ls, "source": source_ls})
res_file = "E:/Workstation/data/NOTAM/myres.xlsx"
df.to_excel(res_file, index=False)

['RWY 30  FOR TKOF AND LDG', "['RWY 30']", 'AVBL', '', 'UNDER FOLLOWING CONDITIONS:X-WIND COMPONENT ON MAIN RWY S EXCEEDS 15 KTS AND ATC IS VISUAL WITH BOTH RWY ENDS RWY 12/30', '', '', '']
['RWY 11/29  ALL TRAINING AND VFR FLIGHTS', "['RWY 11/29']", 'CLOSED', '', '', '', '', '']
['RWY 10R/28L', "['RWY 10R/28L']", 'STRENGTH DECREASED', '', 'SOAKED RWY ARR/DEP WITH AD OPERATOR APPROVAL ONLY TEL +420 602731152', '', '', '']
['RWY 13/31', "['RWY 13/31']", 'NARROWED TO 22,5M', '', 'WHEN WET FOR LDG/TKOF USE SOUTHERNHALF OF RWY', '', '', '']
['RWY 13/31  FOR HELICOPTERS ONLY', "['RWY 13/31']", 'USABLE', '', '', '', '', '']
['REF AIP SUP 12/21 PARAS 23 - 24 AND NOTAM A1171/21,CLOSURE OF CENTRE RWY', '[]', 'IS CNL AND VHHH ON DUAL RWY OPS', 'DUE HKIA CARGO STAND RE-DESIGNATION', 'DRG 252316 -262315', '', '', '']
['RWY 06R/24L', "['RWY 06R/24L']", 'CLSD', '', 'AVBL PPR 30MIN', '', '', '']
['REDL 01/19', '[]', 'U/S', '', 'NIGHT OPS NOT AUTH', '', '', '']
['EDGE LIGHTS, END LIGHTS AND THRESHOLD 

In [7]:
s = "RWY 12/12 ASDAD  RWY 12L/12R ASR RWY 1 RUNWAY 12L RUNWAY12/12"

res = re.findall(r"RWY [0-9]{0,2}[LR]?/?[0-9]{0,2}[LR]?|RUNWAY [0-9]{0,2}[LR]?/?[0-9]{0,2}[LR]?", s, flags=re.I)
str(res)

"['RWY 12/12', 'RWY 12L/12R', 'RWY 1', 'RUNWAY 12L']"

In [8]:
s = "RWY RUNWAY"
p = r"(RUNWAY|RWY)"
res = re.findall(p, s, flags=re.I)
res

['RWY', 'RUNWAY']

In [9]:
res_file = "E:/Workstation/data/NOTAM/myres.xlsx"
eneity_ls = pd.read_excel(res_file)['entity'].values
eneity_ls

array(['RWY 30  FOR TKOF AND LDG',
       'RWY 11/29  ALL TRAINING AND VFR FLIGHTS', 'RWY 10R/28L',
       'RWY 13/31', 'RWY 13/31  FOR HELICOPTERS ONLY',
       'REF AIP SUP 12/21 PARAS 23 - 24 AND NOTAM A1171/21,CLOSURE OF CENTRE RWY',
       'RWY 06R/24L', 'REDL 01/19',
       'EDGE LIGHTS, END LIGHTS AND THRESHOLD LIGHTS RWY 14/32',
       'APPROACH LIGHTING', 'RUNWAY 07R/25L', 'RWY 05L/23R', 'RWY 11/29',
       'RWY 11/29', 'RUNWAY 06/24 50 PERCENT EDGE AND THRESHOLD LIGHTING',
       'REDL RWY 07/25 FM THR 07 TO TWY D', 'REDL 11L/29R',
       'DUE CONSTRUCTION: ILS Z RWY 32L AND ILS Y RWY 32L APCH: ILS/DME MINIMA:',
       'ILS CAT II AND III APCH RWY 08R AND RWY 26L',
       'ILS RWY 05L ON FLIGHT CALIBRATION', 'ILS RWY 12/30 FLT CLBR',
       'ILS RWY 33R', nan, nan,
       'CALIBRATION OF GLIDE SLOPE RWY -25R FREQ 3344MHZ',
       'PAPI RWY 14/32', 'RCLL RWY 11/29',
       'RWY 03R/21L CENTRE LINE LIGHTS', 'RWY 15/33',
       'RWY 30 TKOF AND TGL', 'RWY 06/24', 'RWY 17/35',
  